# Chapter 2 - Django URLs and Views

## URL Regular expressions

In [ ]:
from django.conf.urls import include, url
from django.views.generic import TemplateView

2.1. Correct precedence for Django url regular expressions


In [ ]:
urlpatterns = [
    url(r'^about/index/', TemplateView.as_view(template_name='index.html')),
    url(r'^about/', TemplateView.as_view(template_name='about.html')),
]


2-2. Wrong precedence for Django url regular expressions

In [ ]:
urlpatterns = [
    url(r'^about/', TemplateView.as_view(template_name='about.html')),
    url(r'^about/index/', TemplateView.as_view(template_name='index.html')),
]


2.3. Exact regular expressions, where url order doesn’t matter


In [ ]:
urlpatterns = [
    url(r'^about/$', TemplateView.as_view(template_name='about.html')),
    url(r'^about/index/$', TemplateView.as_view(template_name='index.html')),
]


## Url Parameters, Extra Options, and Query Strings


In [1]:
from django.shortcuts import render
from coffeehouse.stores import views as stores_views
from coffeehouse.about import views as about_views
from coffeehouse.banners import views as banners_views



2.4. Django url parameter definition for access in templates


In [ ]:
urlpatterns = [
    url(
        r'^drinks/(?P<drink_name>\D+)/', 
        TemplateView.as_view(template_name='drinks/index.html'),
    ),
]


2.5. Django url parameter definition for access in view methods in main urls.py file


In [ ]:
# Project main urls.py

urlpatterns = [
    url(r'^stores/(?P<store_id>\d+)/',stores_views.detail),
]


2.6. Django view method in views.py to access url parameter


In [ ]:
def detail(request, store_id):
    # Access store_id with 'store_id' variable
    return render(request, 'stores/detail.html')


2-7. Django urls with optional parameters leveraging the same view method

In [ ]:
urlpatterns = [
    url(r'^stores/', stores_views.detail),
    url(r'^stores/(?P<store_id>\d+)/', stores_views.detail),
]


2.8. Django view method in views.py with default value

In [ ]:
def detail(request, store_id='1'):
    # Access store_id with 'store_id' variable
    return render(request, 'stores/detail.html')


2.9. Django view method extracting url parameters with request.GET

In [ ]:
def detail(request, store_id='1', location=None):
    # Access store_id param with 'store_id' variable and location param with 'location' variable
    # Extract 'hours' or 'map' value appended to url as
    # ?hours=sunday&map=flash
    hours = request.GET.get('hours', '')
    map = request.GET.get('map', '')
    # 'hours' has value 'sunday' or '' if hours not in url
    # 'map' has value 'flash' or '' if map not in url
    return render(request, 'stores/detail.html')


## Url Consolidation and Modularization


2.10. Django urls.py with no url consolidation


In [ ]:
urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html')), 
    url(r'^about/', about_views.index), 
    url(r'^about/contact/', about_views.contact), 
    url(r'^stores/', stores_views.index), 
    url(r'^stores/(?P<store_id>\d+)/', stores_views.detail, {'location':'headquarters'}), 
]
    

2.11. Django urls.py with include to consolidate urls

In [ ]:
urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html')), 
    url(r'^about/', include('coffeehouse.about.urls')), 
    url(r'^stores/', include('coffeehouse.stores.urls'), {'location':'headquarters'}), 
]


2.12. Django /coffeehouse/about/urls.py loaded via include

In [ ]:
urlpatterns = [
    url(r'^$', about_views.index), 
    url(r'^contact/$', about_views.contact), 
]


2.13. Django urls.py with inline include statements

In [ ]:
store_patterns = [
    url(r'^$', stores_views.index), 
    url(r'^(?P<store_id>\d+)/$', stores_views.detail), 
]
about_patterns = [
    url(r'^$', about_views.index), 
    url(r'^contact/$', about_views.contact), 
]
urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html')), 
    url(r'^about/', include(about_patterns)), 
    url(r'^stores/', include(store_patterns), {'location':'headquarters'}), 
]


## Url Naming and Namespaces

In [ ]:
from django.http import HttpResponsePermanentRedirect
from django.core.urlresolvers import reverse

2.14. Django url using name


In [ ]:
# Definition in urls.py

urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html'), name="homepage")
]


# Definition in view method

def method(request):
    #...
    return HttpResponsePermanentRedirect(reverse('homepage'))


In [ ]:
<!--Definition in template-->

<a href="{% url 'homepage' %}">Back to home page</a>


2.15. Django url with arguments using name


In [ ]:
# Definition in urls.py

urlpatterns = [
    url(
        r'^drinks/(?P<drink_name>\D+)/', 
        TemplateView.as_view(template_name='drinks/index.html'), 
        name="drink"
    ),
]

# Definition in view method

def method(request):
    #...
    return HttpResponsePermanentRedirect(reverse('drink', args=(drink.name,)))


In [ ]:
<!--Definition in template-->

<a href="{% url 'drink' drink.name %}">Drink on sale</a>
<a href="{% url 'drink' 'latte' %}">Drink on sale</a>


2.16. Django urls.py with namespace attribute


In [ ]:
# Main urls.py

urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html'), name="homepage"), 
    url(r'^about/', include('coffeehouse.about.urls', namespace="about")), 
    url(r'^stores/', include('coffeehouse.stores.urls', namespace="stores")), 
]

# About urls.py

urlpatterns = [
    url(r'^$', about_views.index, name="index"), 
    url(r'^contact/$', about_views.contact, name="contact"), 
]

#  Stores urls.py

urlpatterns = [
    url(r'^$', stores_views.index, name="index"), 
    url(r'^(?P<store_id>\d+)/$', stores_views.detail, name="detail"), 
]

# Definition in view method

def method(request):
    #...
    return HttpResponsePermanentRedirect(reverse('about:index'))


In [ ]:
<!--Definition in template-->

<a href="{% url 'stores:index' %}">Back to stores index</a>


2.17. Django urls.py with nested namespace attribute


In [ ]:
# Main urls.py

urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html'), name="homepage"), 
    url(r'^stores/', include('coffeehouse.stores.urls', namespace="stores")), 
]

# Stores urls.py

urlpatterns = [
    url(r'^$', stores_views.index, name="index"), 
    url(r'^(?P<store_id>\d+)/$', stores_views.detail, name="detail"), 
    url(r'^(?P<store_id>\d+)/about/', include('coffeehouse.about.urls', namespace="about")), 
]

# About urls.py

urlpatterns = [
    url(r'^$', about_views.index, name="index"), 
    url(r'^contact/$', about_views.contact, name="contact"), 
]

# Definition in view method

def method(request):
    #...
    return HttpResponsePermanentRedirect(reverse('stores:about:index',  args=(store.id, )))


In [ ]:
<!--Definition in template-->

<a href="{% url 'stores:about:index' store.id %}">See about for {{store.name}}</a>


2.18. Django urls.py with multiple instances of the same app


In [ ]:
# Main urls.py

urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html'), name="homepage"), 
    url(r'^coffeebanners/', include('coffeehouse.banners.urls', namespace="coffee-banners")), 
    url(r'^teabanners/', include('coffeehouse.banners.urls', namespace="tea-banners")), 
    url(r'^foodbanners/', include('coffeehouse.banners.urls', namespace="food-banners")), 
]

#  Banners urls.py

urlpatterns = [
    url(r'^$', banners_views.index, name="index"), 
]

# Definition in banners view method

def method(request):
    #...
    #return HttpResponsePermanentRedirect(reverse('coffee-banners:index'))
    #return HttpResponsePermanentRedirect(reverse('tea-banners:index'))
    return HttpResponsePermanentRedirect(reverse('food-banners:index'))


In [ ]:
<!--Definition in template-->
<a href="{% url 'coffee-banners:index' %}">Coffee banners</a>
<a href="{% url 'tea-banners:index' %}">Tea banners</a>
<a href="{% url 'food-banners:index' %}">Food banners</a>


2.19. Django redirect that leverages app_name to determine url


In [ ]:
# Main urls.py

urlpatterns = [
    url(r'^$', TemplateView.as_view(template_name='homepage.html'), name="homepage"), 
    url(r'^coffeebanners/', include('coffeehouse.banners.urls', namespace="coffee-banners")), 
    url(r'^teabanners/', include('coffeehouse.banners.urls', namespace="tea-banners")), 
    url(r'^foodbanners/', include('coffeehouse.banners.urls', namespace="food-banners")), 
]

# Banners urls.py

app_name = 'banners_adverts'
urlpatterns = [
    url(r'^$', banners_views.index, name="index"), 
]

# Logic inside Banners app

def method(request):
    try:
        # ...
        pass
    except:
        return HttpResponsePermanentRedirect(reverse('banners_adverts:index'))


2.20. Django template link that leverages app_name to determine url


In [ ]:
<!--Definition in template banners/index.html-->

<a href="{% url 'banners_adverts:index' %}">{% url 'banners_adverts:index' %}</a>


## View Method Requests and Responses

In [1]:
from django.template.response import TemplateResponse
from django.http import HttpResponse
from django.template import loader, Context


2.21. Set up dictionary in Django view method for access in template


In [ ]:
def detail(request,store_id='1',location=None):
    # Create fixed data structures to pass to template
    # data could equally come from database queries
    # web services or social APIs
    STORE_NAME = 'Downtown'
    store_address = {'street':'Main #385','city':'San Diego','state':'CA'}
    store_amenities = ['WiFi','A/C']
    store_menu = ((0,''),(1,'Drinks'),(2,'Food'))
    values_for_template = {
        'store_name':STORE_NAME, 
        'store_address':store_address, 
        'store_amenities':store_amenities, 
        'store_menu':store_menu
    }
    return render(request,'stores/detail.html', values_for_template)


2.22. Django view method response alternatives

In [ ]:
values_for_template = dict()

# Option 1)

def detail(request, store_id='1', location=None):
    #...
    return render(request, 'stores/detail.html', values_for_template)

# Option 2)


def detail(request, store_id='1', location=None):
    #...
    return TemplateResponse(request, 'stores/detail.html', values_for_template)

# Option 3)

def detail(request, store_id='1', location=None):
    #...
    response = HttpResponse()
    t = loader.get_template('stores/detail.html')
    c = Context(values_for_template)
    return response.write(t.render(c))


## Response Options for HTTP Status and Content-Type Headers

In [2]:
import datetime as dt

2.23. HTTP Content-type and HTTP Status for Django view method responses

In [ ]:
def method1(request):
    # No method body(s) and only render() example provided for simplicity
    # Returns content type text/plain, with default HTTP 200
    return render(
        request,'stores/menu.csv', 
        values_for_template, 
        content_type='text/plain'
    )

def method2(request):
    # Returns HTTP 404, wtih default text/html
    return render(request, 'custom/notfound.html', status=404)

def method3(request):
    # Returns HTTP 500, wtih default text/html
    return render(request, 'custom/internalerror.html', status=500)

def method4(request):
    # Returns content type application/json, with default HTTP 200
    return render(
        request,
        'stores/menu.json', 
        values_for_template, 
        content_type='application/json'
    )


2.24. Override built-in Django HTTP Status view methods in urls.py



In [ ]:
# Overrides the default 400 handler django.views.defaults.bad_request
handler400 = 'coffeehouse.utils.views.bad_request'

# Overrides the default 403 handler django.views.defaults.permission_denied
handler403 = 'coffeehouse.utils.views.permission_denied'

# Overrides the default 404 handler django.views.defaults.page_not_found
andler404 = 'coffeehouse.utils.views.page_not_found'

# Overrides the default 500 handler django.views.defaults.server_error
handler500 = 'coffeehouse.utils.views.server_error'

urlpatterns = [
    #....
]


2.25. Custom views to override built-in Django HTTP view methods


In [ ]:
def page_not_found(request):
    # Dict to pass to template,  data could come from DB query
    values_for_template = {}
    return render(request, '404.html', values_for_template, status=404)

def server_error(request):
    # Dict to pass to template,  data could come from DB query
    values_for_template = {}
    return render(request, '500.html', values_for_template, status=500)

def bad_request(request):
    # Dict to pass to template,  data could come from DB query
    values_for_template = {}
    return render(request, '400.html', values_for_template, status=400)

def permission_denied(request):
    # Dict to pass to template,  data could come from DB query
    values_for_template = {}
    return render(request, '403.html', values_for_template, status=403)


2.26. HttpResponse with template and custom CSV file download

In [ ]:

def view_method():
    sorted_users = list()
    response = HttpResponse(content_type='text/csv')
    response['Content-Disposition'] = 'attachment; filename=Users_{}.csv'\
        .format(dt.datetime.now())
    t = loader.get_template('dashboard/users_csvexport.html')
    c = Context({'users': sorted_users,})
    response.write(t.render(c))
    return response


## View Method Middleware

2.27. Default Django middleware classes in MIDDLEWARE

In [ ]:
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

2.28. Django middleware class structure

In [ ]:
class CoffeehouseMiddleware(object):
    def __init__(self, get_response):
        self.get_response = get_response

    # One-time configuration and initialization on start-up
    def __call__(self, request):
        # Logic executed on a request before the view (and other middleware) is called.
        # get_response call triggers next phase
        response = self.get_response(request)
        # Logic executed on response after the view is called.
        # Return response to finish middleware sequence
        return response

    def process_view(self, request, view_func, view_args, view_kwargs):
        # Logic executed before a call to view
        # Gives access to the view itself & arguments
        return

    def process_exception(self,request, exception):
        # Logic executed if an exception/error occurs in the view
        return

    def process_template_response(self,request, response):
        # Logic executed after the view is called,
        # ONLY IF view response is TemplateResponse, see listing 2-22
        return
